<a href="https://colab.research.google.com/github/AAhmadS/NLP_HW2/blob/main/phonetics%20foreign%20word%20detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/NLP

/content/drive/.shortcut-targets-by-id/1Ikn75GANDVFyb0klJe4-EBQJikt4_Ob_/NLP


##Extracting essential data



###Essential packages

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
#installing the necessary libraries
!pip install -q hazm
!pip install -q dadmatools
!pip install -q googletrans==3.1.0a0

In [ ]:
#importing the necessary libraries
from __future__ import unicode_literals
import hazm

import dadmatools.pipeline.language as language

import json
import pickle
import itertools
from tqdm import tqdm
import pandas as pd

###Creating dataset

In [ ]:
with open("Nlp HW2/corpus.txt","rb") as f:
  corpo = pickle.load(f)

corpo

['آئین',
 'آب',
 'آبا',
 'آباجی',
 'آباد',
 'آبادسازی',
 'آباده',
 'آبادکننده',
 'آبادگر',
 'آبادی\u200cنشین',
 'آبان',
 'آباژور',
 'آباژورساز',
 'آباژورسازی',
 'آبتنی',
 'آبتین',
 'آبجو',
 'آبجوسازی',
 'آبجی',
 'آبخازستان',
 'آبخازیا',
 'آبخوان',
 'آبخور',
 'آبخوری',
 'آبخیز',
 'آبخیزداری',
 'آبخیزگاه',
 'آبدار',
 'آبدارخانه',
 'آبدارچی',
 'آبدزدک',
 'آبدیده',
 'آبدیدگی',
 'آبراه',
 'آبراهام',
 'آبراهه',
 'آبرفت',
 'آبرنگ',
 'آبرو',
 'آبروبری',
 'آبرودار',
 'آبروداری',
 'آبروریزی',
 'آبرومند',
 'آبرومندانه',
 'آبریز',
 'آبریزگاه',
 'آبزی',
 'آبزی\u200cپروری',
 'آبسال',
 'آبستراکسیون',
 'آبستره',
 'آبستن',
 'آبستنی',
 'آبسه',
 'آبسکون',
 'آبشار',
 'آبشامه',
 'آبشخور',
 'آبعلی',
 'آبله',
 'آبله\u200cدار',
 'آبله\u200cرو',
 'آبله\u200cمرغان',
 'آبله\u200cکوبی',
 'آبله\u200cگون',
 'آبنوس',
 'آبونمان',
 'آبپاش',
 'آبکار',
 'آبکاری',
 'آبکش',
 'آبکی',
 'آبگرم',
 'آبگوشت',
 'آبگوشت\u200cخور',
 'آبگوشت\u200cخوری',
 'آبگون',
 'آبگیر',
 'آبگینه',
 'آبیاری',
 'آبی\u200cپوش',
 'آب\u200cانبار',
 '

In [ ]:
#the corpus used for this task comes from hazm word list
corpus = list(set(corpo))
len(corpus)

35316

In [ ]:
# !pip install sentencepiece
# !pip install transformers
# from transformers import AutoTokenizer
# from transformers import AutoModelForTokenClassification  # for pytorch
# from transformers import TFAutoModelForTokenClassification  # for tensorflow
# from transformers import pipeline
# # model_name_or_path = "HooshvareLab/bert-fa-zwnj-base-ner"  # Roberta
# # model_name_or_path = "HooshvareLab/roberta-fa-zwnj-base-ner"  # Roberta
# model_name_or_path = "HooshvareLab/distilbert-fa-zwnj-base-ner"  # Distilbert
# # model_name_or_path = "HooshvareLab/albert-fa-zwnj-base-v2-ner"  # Albert

# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# # tokenizer=hazm.WordTokenizer
# model = AutoModelForTokenClassification.from_pretrained(model_name_or_path)  # Pytorch
# # model = TFAutoModelForTokenClassification.from_pretrained(model_name_or_path)  # Tensorflow

# ner = pipeline("ner", model=model, tokenizer=tokenizer)


In [ ]:
len(corpus)

35316

In [ ]:
# persian_names=pd.read_csv("Nlp HW2/Persian names.csv")
# persian_names_list=persian_names["first_name"]
# persian_names_list=persian_names_list.tolist()
# for word in corpus:
#   ner_res=ner(word)
#   if len(ner_res)==0:
#     continue
#   if ner(word)[0]["entity"]=="B-PER" and word not in persian_names_list:
#     print(word)
#     corpus.remove(word)


In [ ]:
len(corpus)

35316

In [ ]:
# file = open('Nlp HW2/corpus_without_english_names.txt','w',encoding = 'utf-8')
# for word in corpus:
#   file.write(word)
#   file.write("\n")
# file.close()

In [ ]:
#roles contains all the possible roles in our dictionary
roles = [x[2] for x in hazm.words_list()]
roles = list(set(itertools.chain(*roles)))
roles

['AJC',
 'NUM',
 'CL',
 'ZVR',
 'POSTP',
 'RES',
 'V',
 'COMP',
 'PL',
 'INT',
 'PS',
 'PR',
 'P',
 'NIN',
 'PRO',
 'N',
 'CONJ',
 'DET',
 'ADV',
 'AJ']

In [ ]:
lemmatizer = hazm.Lemmatizer()
stemmer = hazm.Stemmer()

In [ ]:
'''this way of defining stop_words may not be usual,
but the main purpose has been to eliminate disturbing words;
such as: punctiation marks, pronouns and Persian\Arabic grammatical signs'''

stop_words = ["ند","یم","ید","ی","تان","ش","ت","شان","مان","م","ها","ات","ان","ین","ون","‌های","ام","ای"]
stop_words.extend([".","؛",",",":","?","!","؟","ْ","ٌ","ٍ","ً","ُ","ِ","َ","ّ","}","ؤ","{","»","«","ٰ","‌‌‌‌‌‌‌‌ٔ","ء","\"","(",")"])
more_stopwords=["ها","تر","ترین"]
stop_words.extend(more_stopwords)

In [ ]:
add_list = ["ی","تان","ش","ت","شان","مان","م","‌ها"]
adj_add_list = ["تر","ترین"]
corpu = []
for word in hazm.words_list():
  corpu.append(word[0])
  corpu.append(lemmatizer.lemmatize(word[0]))
  if ("N" in word[2]) or ("AJC" in word[2]) or ("AJ" in word[2]):
    for ele in add_list:
      corpu.append(word[0]+ele)
  if ("AJC" in word[2]) or ("AJ" in word[2]):
    for ele in adj_add_list:
      corpu.append(word[0]+ele)

corpu.append("آره")
corpu = list(set(corpu))
len(corpu)

336122

##Main code

###Config

In [ ]:
#nlp is the dadmatools' pipeline used for lemmatization purpose only
pips = 'lem'
nlp = language.Pipeline(pips)

print(nlp.analyze_pipes(pretty=True))

Model fa_tokenizer exists in /root/.pernlp/fa_tokenizer.pt
Model fa_mwt exists in /root/.pernlp/fa_mwt.pt
Model fa_lemmatizer exists in /root/.pernlp/fa_lemmatizer.pt

============================= Pipeline Overview =============================

#   Component   Assigns       Requires   Scores   Retokenizes
-   ---------   -----------   --------   ------   -----------
0   tokenizer                                     True       
                                                             
1   lemmatize   token.lemma                       False      

✔ No problems found.
{'summary': {'tokenizer': {'assigns': [], 'requires': [], 'scores': [], 'retokenizes': True}, 'lemmatize': {'assigns': ['token.lemma'], 'requires': [], 'scores': [], 'retokenizes': False}}, 'problems': {'tokenizer': [], 'lemmatize': []}, 'attrs': {'token.lemma': {'assigns': ['lemmatize'], 'requires': []}}}


In [ ]:
#we use an args dictionary to initialize our model
args = {
    'corpus':corpu,
    'pipe':nlp,
    'stop_words':stop_words,
    'normalizer':hazm.Normalizer(),
    'lemmatizer':hazm.Lemmatizer(),
    'POS_tagger':hazm.POSTagger(model='Nlp HW2/resources/postagger.model'),
    'translator':None,
    'word_tokenizer':hazm.WordTokenizer(),
    'sent_tokenizer':hazm.SentenceTokenizer()
}

###Model

In [ ]:
#as the name suggests, the class is used to spot and show the positin of foreign words within the text

class foreign_word_detector():

  def __init__(
      self,
      args
      ):

    self.corpus = args['corpus']
    self.stop_words = args['stop_words']
    self.lang_pipe = args['pipe']
    self.normalizer = args['normalizer']
    self.lemmatizer = args['lemmatizer']
    self.translator = args['translator']
    self.word_tokenizer = args['word_tokenizer']
    self.sent_tokenizer = args['sent_tokenizer']
    self.tagger = args['POS_tagger']

#the following function is the main funtion of the model used for detecting the foreign words and their position
  def detect(self, text):
    sentenced = self.sent_tokenizer.tokenize(text)

    normalized = [self.normalizer.normalize(sent) for sent in sentenced]
    normalized = [sent.replace("\u200cهای","") for sent in normalized]
    normalized = [sent.replace("\u200c"," ") for sent in normalized]
    # print(f"normalized:{normalized}")
    lemmed = [str(self.lang_pipe(sent)) for sent in normalized]
    # print(f"lemmed:{lemmed}")
    words_list = [self.word_tokenizer.tokenize(sent) for sent in lemmed]

    final_words_list = []
    for sent in words_list:
      words_second_list = []
      tags = self.tagger.tag(sent)
      for i in range(len(tags)):
        if tags[i][1] != "V":
          words_second_list.append(tags[i][0])
      final_words_list.append(words_second_list)

    words_list = list(itertools.chain(*final_words_list))
    words_list = [word for word in words_list if word not in self.stop_words]
    #lemmed_words_list = [lemmatizer.lemmatize(word) for word in words_list]
    # print(lemmed_words_list)

    output = dict()
    for word in words_list:
      if word not in self.corpus:
        output[word] = []
        add_ind = 0

        string = text
        while word in string:

          begin = string.find(word)
          output[word].append((begin+add_ind,add_ind+begin+len(word)))

          string = string[begin+len(word):]
          add_ind+= begin+len(word)

    return output


###Test

####results

In [ ]:
dataframe = pd.read_csv("Nlp HW2/test_file.csv")

In [ ]:
dataframe.fillna("",inplace=True)

In [ ]:
#testing the model:
detector = foreign_word_detector(args)
true_count = 0
tp_count = 0
fp_count = 0
fn_count = 0
for i in range(dataframe.shape[0]):
  sample = dataframe["sample"][i]
  words = dataframe["word"][i].split()
  results = detector.detect(sample).keys()
  for word in results:
    if word in words:
      tp_count+=1
    else:
      fp_count+=1
  true_count+=len(words)

fn_count = true_count - tp_count

print(f"true_positive_count = {tp_count}\nfalse_positive_count = {fp_count}\nfalse_negative_count = {fn_count}\nprecision = {tp_count/(tp_count+fp_count) :.2f}\nrecall = {tp_count/(true_count):.2f}")


true_positive_count = 103
false_positive_count = 8
false_negative_count = 19
precision = 0.93
recall = 0.84


####user run

In [ ]:
#the run function, is an interactive way to map user_input to the detectors answer
def run():
  detector = foreign_word_detector(args)

  print("enter your sample text.\nenter an empty string if you want to end the process")
  while True :
    text = input()
    if text == "":
      break

    output = detector.detect(text)
    print(output)


In [ ]:
run()

##New idea

###Translate and replace

In [ ]:
def change_english_words(text):
  from googletrans import Translator
  translator = Translator()
  detector = foreign_word_detector(args)
  output = detector.detect(text)
  print(f"Output: {output}")
  english_text=text
  new_text=text
  for word in output.keys():
    # first_index=output[word][0]
    # second_index=output[word][1]
    english_form=translator.translate(word).text
    persian_translation=translator.translate(english_form, dest='fa').text
    persian_translation=persian_translation.split("-")[0]
    new_text=new_text.replace(word,persian_translation)
  return new_text,english_text

In [ ]:
# text= "تسک سختی بود ولی تو کانتریبیوشن خوبی داشتی تنکس ."
text= "سلام من و فرزندت در گاردنمان همبرگر و استیک خوریدم و آن‌ ها خیلی دلیشز بودند ."
# text="من و فرندم نشستیم گیم آف ترونز دیدیم ولی اصلا به گرد پای برکینگ بد هم نمیرسید."
translation,english_text=change_english_words(text)
print(translation)
# print(english_text)

Output: {'گاردن': [(20, 25)], 'دلیشز': [(65, 70)]}
سلام من و فرزندت در باغمان همبرگر و استیک خوریدم و آن‌ ها خیلی خوشمزه  بودند .


###Using phonetics

In [ ]:
#We import an English dictionary. Add the english words in "words" and the corresponding phonetics to "phonics_list".
words = []
phonics_list = []
dict_file = open('Nlp HW2/cmudict.dict', 'r')
with dict_file as f:
    phonics = [line.rstrip('\n') for line in f]
    for p in phonics:
        x = p.split(' ')
        words.append(x[0])
        phonics_list.append(' '.join(x[1:]))

In [ ]:
phons_unique = list(set(itertools.chain(*[ph.split(" ") for ph in phonics_list])))
phons_as_list = [ph.split(" ") for ph in phonics_list]

In [ ]:
with open("Nlp HW2/phons_exchange.txt","rb") as file:
  phons_dict = pickle.load(file)

In [ ]:
phons_dict["NG"] = ("ن")
phons_dict["TH"] = ("ت")
phons_dict["AO2"] = phons_dict["AO1"]
phons_dict["IH2"] = ("ای","ی")
phons_dict["IH0"] = phons_dict["IH2"]
phons_dict["IH1"] = phons_dict["IH0"]
phons_dict["IY2"] = ("ای","ی")
phons_dict["UH0"] = ("او","و")
phons_dict["UH2"] = phons_dict["UH0"]
phons_dict["UH1"] = phons_dict["UH0"]
phons_dict["IY1"] = phons_dict["IY2"]
phons_dict["IY0"] = phons_dict["IY2"]

In [ ]:
#Converting our words to their persian correspondent with the use of phonetics in the "persianized"
persianized_list = []
persianized = []
for x in phons_as_list:
  word_list = []
  count=0
  for ph in x:
    count+=1
    if len(phons_dict[ph])>1:
      if count==1:
        word_list.append(phons_dict[ph][0])
      else:
        word_list.append(phons_dict[ph][1])
    else:
      word_list.append(phons_dict[ph][0])

  persianized_list.append(word_list)
  persianized.append(''.join(word_list))

In [ ]:
#Checking some words from "words"
words[41000:41010]

['fancier',
 'fanciers',
 'fancies',
 'fanciest',
 'fanciful',
 'fancy',
 'fandango',
 'fandel',
 'fandrich',
 'fane']

In [ ]:
#Checking the corresponding persionized words from "persionized"
persianized[41000:41010]

['فنسیر',
 'فنسیرز',
 'فنسیز',
 'فنسیست',
 'فنسیفل',
 'فنسی',
 'فندنگو',
 'فندل',
 'فندریک',
 'فین']

In [ ]:
#Checking if our persionized method works fine on some words.
listt = ['contribution','thanks','computer','system','task','time','garden','burger','chop','fandrich','infinity']

for i in range(len(words)):
  if words[i] in listt:
    print(words[i])
    print(phons_as_list[i])
    print(i)
    print(persianized[i])
    print("-----------------")

burger
['B', 'ER1', 'G', 'ER0']
16631
برگر
-----------------
chop
['CH', 'AA1', 'P']
21283
چاپ
-----------------
computer
['K', 'AH0', 'M', 'P', 'Y', 'UW1', 'T', 'ER0']
23974
کمپیوتر
-----------------
contribution
['K', 'AA2', 'N', 'T', 'R', 'AH0', 'B', 'Y', 'UW1', 'SH', 'AH0', 'N']
25006
کانتربیوشن
-----------------
fandrich
['F', 'AE1', 'N', 'D', 'R', 'IH0', 'K']
41008
فندریک
-----------------
garden
['G', 'AA1', 'R', 'D', 'AH0', 'N']
46458
گاردن
-----------------
infinity
['IH2', 'N', 'F', 'IH1', 'N', 'IH0', 'T', 'IY0']
59556
اینفینیتی
-----------------
system
['S', 'IH1', 'S', 'T', 'AH0', 'M']
118902
سیستم
-----------------
task
['T', 'AE1', 'S', 'K']
119638
تسک
-----------------
thanks
['TH', 'AE1', 'NG', 'K', 'S']
120780
تنکس
-----------------
time
['T', 'AY1', 'M']
121710
تایم
-----------------


In [ ]:
#phons_dict contains the Persian equivalent of English phonetics
phons_dict

{'EH0': ('ا', ''),
 'ER2': ('ار', 'ر'),
 'old': ('',),
 'EY0': ('ای', 'ی'),
 'IY2': ('ای', 'ی'),
 'name': ('',),
 'S': ('س',),
 'CH': ('چ',),
 'K': ('ک',),
 'UW2': ('یو',),
 'AA0': ('آ', 'ا'),
 'AY2': ('آی', 'ای'),
 'OY0': ('اوی', 'وی'),
 'AA1': ('آ', 'ا'),
 'L': ('ل',),
 'UH0': ('او', 'و'),
 'B': ('ب',),
 'F': ('ف',),
 'OY1': ('اوی', 'وی'),
 'abbrev': ('',),
 'UH2': ('او', 'و'),
 'D': ('د',),
 'JH': ('ج',),
 'W': ('و',),
 'G': ('گ',),
 'IH0': ('ای', 'ی'),
 'UH1': ('او', 'و'),
 'french': ('',),
 'Z': ('ز',),
 'AA2': ('آ', 'ا'),
 'OY2': ('اوی', 'وی'),
 'SH': ('ش',),
 'AH0': ('ا', ''),
 'foreign': ('',),
 'UW0': ('یو',),
 'AH1': ('ا', ''),
 'IH1': ('ای', 'ی'),
 'EY2': ('ای', 'ی'),
 'AH2': ('ا', ''),
 'OW2': ('او', 'و'),
 'Y': ('ی',),
 'AE2': ('',),
 'DH': ('ض',),
 'P': ('پ',),
 'ER1': ('ار', 'ر'),
 'AW2': ('او',),
 '#': ('',),
 'EH1': ('ا', ''),
 'AE1': ('',),
 'EH2': ('ا', ''),
 'N': ('ن',),
 'ZH': ('ز',),
 'OW1': ('او', 'و'),
 'UW1': ('او', 'و'),
 'AY0': ('آی', 'ای'),
 'AY1': ('آی', 'ا

In [ ]:
!pip install -U strsimpy
import pandas as pd
from strsimpy.weighted_levenshtein import WeightedLevenshtein


def insertion_cost(char):
  if char=="ا":
    return 0
  return 1.0


def deletion_cost(char):
  if char=="ا":
    return 0
  return 1.0


def substitution_cost(char_a, char_b):
  if char_a=="ا" and char_b=="آ":
    return 0
  return 2.0

weighted_levenshtein = WeightedLevenshtein(
    substitution_cost_fn=substitution_cost,
    insertion_cost_fn=insertion_cost,
    deletion_cost_fn=deletion_cost)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
w="هاردی"
for i,word in enumerate(persianized):
  if word==w:
    print(word)
    print(words[i])

هاردی
hardee
هاردی
hardey
هاردی
hardie
هاردی
hardy


In [ ]:
w="هاردی"
for i in hazm.words_list():
  if i[0]==w:
    print(i)

('هاردی', 4, ('N',))


In [ ]:
w1="کامپیوتر"
w2="کمپیوتر"
for i in persianized:
  if i==w2:
    print(i)
weighted_levenshtein.distance(w1,w2)

کمپیوتر


0

In [ ]:
from hazm import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from googletrans import Translator

# Example sentences in Persian
''' For words that are both Persian and English, we use the PickSentence method that chooses between two options. First option is the persian version and the second
option is the english version. We check which sentence makes more sense and choose that sentence according to cosine similarity.
'''
intersected = list(set(persianized) & set(corpu))
args = {
    'corpus':corpu,
    'pipe':nlp,
    'stop_words':stop_words,
    'intersected':intersected,
    'normalizer':hazm.Normalizer(),
    'lemmatizer':hazm.Lemmatizer(),
    'POS_tagger':hazm.POSTagger(model='Nlp HW2/resources/postagger.model'),
    'translator':Translator(),
    'word_tokenizer':hazm.WordTokenizer(),
    'sent_tokenizer':hazm.SentenceTokenizer()
}

In [ ]:
#as the name suggests, the class is used to spot and show the positin of foreign words within the text
class phonetics_foreign_word_detector():


  def __init__(
      self,
      args
      ):

    self.corpus = args['corpus']
    self.stop_words = args['stop_words']
    self.intersected = args['intersected']
    self.lang_pipe = args['pipe']
    self.normalizer = args['normalizer']
    self.lemmatizer = args['lemmatizer']
    self.translator = args['translator']
    self.word_tokenizer = args['word_tokenizer']
    self.sent_tokenizer = args['sent_tokenizer']
    self.tagger = args['POS_tagger']

  def is_foreign_word(self,word):
    for english_word in persianized:
      if weighted_levenshtein.distance(word,english_word)==0:
        return True
    return False


#the following function is the main funtion of the model used for detecting the foreign words and their position
  def detect(self, text):
    sentenced = self.sent_tokenizer.tokenize(text)

    normalized = [self.normalizer.normalize(sent) for sent in sentenced]
    normalized = [sent.replace("\u200cهای","") for sent in normalized]
    normalized = [sent.replace("\u200c"," ") for sent in normalized]
    # print(f"normalized:{normalized}")
    lemmed = [str(self.lang_pipe(sent)) for sent in normalized]
    # print(f"lemmed:{lemmed}")
    words_list = [self.word_tokenizer.tokenize(sent) for sent in lemmed]

    final_words_list = []
    for sent in words_list:
      words_second_list = []
      tags = self.tagger.tag(sent)
      for i in range(len(tags)):
        if tags[i][1] != "V":
          words_second_list.append(tags[i][0])
      final_words_list.append(words_second_list)

    words_list = list(itertools.chain(*final_words_list))
    words_list = [word for word in words_list if word not in self.stop_words]
    #lemmed_words_list = [lemmatizer.lemmatize(word) for word in words_list]
    # print(lemmed_words_list)

    output = dict()
    for word in words_list:
      if word in self.intersected:
        option1 = word

        english_form=self.translator.translate(option1).text
        option2=self.translator.translate(english_form, dest='fa').text

        sentence1= text
        sentence2 = text.replace(option1,option2)

        _,chosen_word = self.PickSentence(sentence1, sentence2, option1,option2)

        if chosen_word == option2:
          output[word] = []
          add_ind = 0

          string = text
          while word in string:

            begin = string.find(word)
            output[word].append((begin+add_ind,add_ind+begin+len(word)))

            string = string[begin+len(word):]
            add_ind+= begin+len(word)

      else:
        if self.is_foreign_word(word):
          output[word] = []
          add_ind = 0

          string = text
          while word in string:

            begin = string.find(word)
            output[word].append((begin+add_ind,add_ind+begin+len(word)))

            string = string[begin+len(word):]
            add_ind+= begin+len(word)

    return output

  def PickSentence(self,sentence1, sentence2, option1,option2):

    '''
    sentence1 = "من گرسنه هستم. آب میخواهم."
    sentence2 = "من تشنه هستم. آب میخواهم."

    Candidate words
    option1 = "گرسنه"
    option2 = "تشنه"
    '''

    # Tokenize the sentences
    tokens1 = word_tokenize(sentence1)
    tokens2 = word_tokenize(sentence2)

    # Identify the blank space in each sentence
    blank_index1 = tokens1.index(option1)
    blank_index2 = tokens2.index(option2)

    # Contextualize each sentence without the blank space
    context_tokens1 = tokens1[:blank_index1] + ["____"] + tokens1[blank_index1+1:]
    context1 = ' '.join(context_tokens1)
    context_tokens2 = tokens2[:blank_index2] + ["____"] + tokens2[blank_index2+1:]
    context2 = ' '.join(context_tokens2)

    # Use a language model to predict which sentence is more likely
    normalizer = Normalizer()
    doc1 = normalizer.normalize(sentence1)
    context1 = normalizer.normalize(context1)
    doc2 = normalizer.normalize(sentence2)
    context2 = normalizer.normalize(context2)

    stemmer = Stemmer()
    doc_stemmed1 = ' '.join([stemmer.stem(word) for word in word_tokenize(doc1)])
    context_stemmed1 = ' '.join([stemmer.stem(word) for word in word_tokenize(context1)])
    doc_stemmed2 = ' '.join([stemmer.stem(word) for word in word_tokenize(doc2)])
    context_stemmed2 = ' '.join([stemmer.stem(word) for word in word_tokenize(context2)])

    # Calculate cosine similarity score for each option
    vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
    X = vectorizer.fit_transform([doc_stemmed1, context_stemmed1, doc_stemmed2, context_stemmed2])
    cosine_similarities1 = cosine_similarity(X)[0][1]
    cosine_similarities2 = cosine_similarity(X)[2][3]

    # Choose the sentence with the higher score
    if cosine_similarities1 > cosine_similarities2:
        chosen_sentence = sentence1
        chosenword=option1

    else:
        chosen_sentence = sentence2
        chosenword=option2

    return chosen_sentence, chosenword

In [ ]:
#the run function, is an interactive way to map user_input to the detectors answer
def phonetics_run():
  detector = phonetics_foreign_word_detector(args)

  print("enter your sample text.\nenter an empty string if you want to end the process")
  while True :
    text = input()
    if text == "":
      break

    output = detector.detect(text)
    print(output)


In [ ]:
phonetics_run()